In [ ]:
from collections import defaultdict
from copy import deepcopy
from datetime import datetime, timedelta
import matplotlib.dates as mdates
from matplotlib.pyplot import *
from os import listdir
import pickle

In [ ]:
#data = pickle.load(open('pickles/scenario1_r02.p', 'rb'))
#sources, distribution, types, summary = data

def get_data(scenario):
    return pickle.load(open('pickles/' + scenario + '.p', 'rb'))

In [ ]:
def exposure_curve(summary, save = None):
    #return
    locator = mdates.AutoDateLocator(interval_multiples = False)
    formatter = mdates.AutoDateFormatter(locator)
    fig, ax1 = subplots()
    T = summary['T']
    Es = summary['Es']
    ax1.plot(T, [np.mean(Es[t]) for t in T])
    ax1.set_title('Epidemic Progression')
    ax1.set_xlabel('Day of Year')
    ax1.set_ylabel('Number of Students')
    ax1.xaxis.set_major_locator(locator) # Annoying code to make dates work!
    ax1.xaxis.set_major_formatter(formatter)
    fig.autofmt_xdate()
    tight_layout()
    if save:
        fig.savefig(save + '-exposures.pdf')
    show()

def sources_bars(sources, save = None):
    fig, ax1 = subplots()
    xs = []
    ys = []
    for key, value in sources.items():
        xs.append(key)
        ys.append(value)

    ax1.bar(xs, ys)
    ax1.set_title('Sources of Infections')
    ax1.set_ylabel('Number of People')
    xticks(rotation = 30)
    tight_layout()
    if save:
        fig.savefig(save + '-sources.pdf')
    show()

def boxplot_totals(scenario_distributions, save = None):
    scenario_totals = defaultdict(list)
    for scenario in scenario_distributions:
        for counts in scenario_distributions[scenario].values():
            items = len(counts)
            break
        for i in range(items):
            total = 0
            for counts in scenario_distributions[scenario].values():
                total += counts[i]
            scenario_totals[scenario].append(total)
    
    xs = []
    ys = []
    for scenario in sorted(scenario_totals):
        xs.append(scenario)
        ys.append(scenario_totals[scenario])
    
    boxplot(ys)
    xticks(np.array(range(len(xs))) + 1, xs, rotation = 30)
    ylabel('Number of People')
    title('Distribution of Infection Counts by Scenario')
    tight_layout()
    if save:
        gcf().savefig(save + '-counts.pdf')
    show()

def boxplot_community(scenario_distributions, save = None):
    scenario_totals = defaultdict(list)
    for scenario in scenario_distributions:
        scenario_totals[scenario] = scenario_distributions[scenario]['Community']
    
    xs = []
    ys = []
    for scenario in sorted(scenario_totals):
        xs.append(scenario)
        ys.append(scenario_totals[scenario])
    
    boxplot(ys)
    xticks(np.array(range(len(xs))) + 1, xs, rotation = 30)
    ylabel('Number of People')
    title('Community Source Infection Counts by Scenario')
    tight_layout()
    if save:
        gcf().savefig(save + '-community_sources.pdf')
    show()

def stackedbar_sources(scenario_sources, plottitle, save = None):
    source_list = set()
    for ss in scenario_sources.values():
        source_list.update(set(ss))
    
    source_vectors = defaultdict(list)
    for name, ss in scenario_sources.items():
        for s, value in ss.items():
            source_vectors[s].append(value)
    
    at_level = [0 for x in range(len(scenario_sources))]
    for source in sorted(source_list):
        bar(list(scenario_sources.keys()), source_vectors[source], .5, label = source, bottom = at_level)
        for i, x in enumerate(source_vectors[source]):
            at_level[i] += x

    legend()
    xticks(rotation = 30)
    ylabel('Number of People')
    title(plottitle)
    tight_layout()
    if save:
        gcf().savefig(save + '-sources.pdf')
    show()

In [ ]:
def summarize(file):
    ''' Need mean, median, and standard deviation for each scenario. '''
    sources, distribution, types, summary = get_data(file[:-2])
    counts = defaultdict(int)
    for place, count in distribution.items():
        for i, n in enumerate(count):
            counts[i] += n
    
    counts = np.array(list(counts.values()))
    return ['"' + file[:-2] + '"', np.mean(counts), np.median(counts), np.std(counts, ddof=1)]

In [ ]:
#prefix = 'Results 2021 04 11/pickles'
with open('SummaryStats.csv', 'w') as fout:
    fout.write('Scenario,Mean,Median,Stdev\n')
    for file in sorted(listdir('pickles')):
        fout.write(','.join(map(str, summarize(file))))
        fout.write('\n')

In [ ]:
listdir(prefix)

In [ ]:
# Scenario 1, R0 = 2.
sources, distribution, types, summary = get_data('scenario1_r02')
exposure_curve(summary, save='rerendered/scenario1-R0=2')
sources_bars(sources, save='rerendered/scenario1-R0=2')

In [ ]:
# Scenario 1, R0 = 3.
sources, distribution, types, summary = get_data('scenario1_r03')
exposure_curve(summary, save='rerendered/scenario1-R0=3')
sources_bars(sources, save='rerendered/scenario1-R0=3')

In [ ]:
# Scenario 1, R0 = 4.
sources, distribution, types, summary = get_data('scenario1_r04')
exposure_curve(summary, save='rerendered/scenario1-R0=4')
sources_bars(sources, save='rerendered/scenario1-R0=4')

In [ ]:
# Scenario 2.
sources, distribution, types, summary = get_data('scenario2')
exposure_curve(summary, save='rerendered/scenario2')
sources_bars(sources, save='rerendered/scenario2')

In [ ]:
# Scenario 3, R0 = 2.
datas = defaultdict(dict)
for v in [.5, .75]:
    for ri, ro in [(.3, .3), (.3, .7), (.7, .3), (.7, .7)]:
        name = 'V' + str(v) + ',' + str((ri, ro))
        sources, distribution, types, summary = get_data('scenario3_' + name + '_r02')
        datas['sources'][name] = deepcopy(sources)
        datas['distribution'][name] = deepcopy(distribution)
        datas['types'][name] = deepcopy(types)
        

boxplot_totals(datas['distribution'], save='rerendered/scenario3-R0=2')
boxplot_community(datas['distribution'], save='rerendered/scenario3-R0=2')
stackedbar_sources(datas['sources'], 'Source of Infections by Scenario', save='rerendered/scenario3-R0=2')
stackedbar_sources(datas['types'], 'Vaccination Status of Infected Individuals by Scenario', save='rerendered/scenario3-R0=2-vaccine')

In [ ]:
# Scenario 3, R0 = 3.
datas = defaultdict(dict)
for v in [.5, .75]:
    for ri, ro in [(.3, .3), (.3, .7), (.7, .3), (.7, .7)]:
        name = 'V' + str(v) + ',' + str((ri, ro))
        sources, distribution, types, summary = get_data('scenario3_' + name + '_r03')
        datas['sources'][name] = deepcopy(sources)
        datas['distribution'][name] = deepcopy(distribution)
        datas['types'][name] = deepcopy(types)
        

boxplot_totals(datas['distribution'], save='rerendered/scenario3-R0=3')
boxplot_community(datas['distribution'], save='rerendered/scenario3-R0=3')
stackedbar_sources(datas['sources'], 'Source of Infections by Scenario', save='rerendered/scenario3-R0=3')
stackedbar_sources(datas['types'], 'Vaccination Status of Infected Individuals by Scenario', save='rerendered/scenario3-R0=3-vaccine')

In [ ]:
# Scenario 3, R0 = 4.
datas = defaultdict(dict)
for v in [.5, .75]:
    for ri, ro in [(.3, .3), (.3, .7), (.7, .3), (.7, .7)]:
        name = 'V' + str(v) + ',' + str((ri, ro))
        sources, distribution, types, summary = get_data('scenario3_' + name + '_r04')
        datas['sources'][name] = deepcopy(sources)
        datas['distribution'][name] = deepcopy(distribution)
        datas['types'][name] = deepcopy(types)
        

boxplot_totals(datas['distribution'], save='rerendered/scenario3-R0=4')
boxplot_community(datas['distribution'], save='rerendered/scenario3-R0=4')
stackedbar_sources(datas['sources'], 'Source of Infections by Scenario', save='rerendered/scenario3-R0=4')
stackedbar_sources(datas['types'], 'Vaccination Status of Infected Individuals by Scenario', save='rerendered/scenario3-R0=4-vaccine')

In [ ]:
# Scenario 3, R0 = 3 verification of statistics.
datas = defaultdict(dict)
for v in [.5, .75]:
    for ri, ro in [(.3, .3), (.3, .7), (.7, .3), (.7, .7)]:
        name = 'V' + str(v) + ',' + str((ri, ro))
        sources, distribution, types, summary = get_data('scenario3_' + name + '_r03')
        print(np.mean(distribution['Socialization']))
        totalsd = []
        totalss = []
        for i in range(100):
            totalsd.append(sum([vector[i] for vector in distribution.values()]))
            totalss.append(sum(sources.values()))
        print(np.mean(totalsd), np.mean(totalss))
        


In [ ]:
# Scenario 4.
datas = defaultdict(dict)
for v, ri, ro in [(.5, .3, .3), (.75, .7, .7)]:
    name = 'V' + str(v) + ',' + str((ri, ro))
    sources, distribution, types, summary = get_data('scenario4_' + name)
    datas['sources'][name] = deepcopy(sources)
    datas['distribution'][name] = deepcopy(distribution)
    datas['types'][name] = deepcopy(types)
        

boxplot_totals(datas['distribution'], save='rerendered/scenario4')
boxplot_community(datas['distribution'], save='rerendered/scenario4')
stackedbar_sources(datas['sources'], 'Source of Infections by Scenario', save='rerendered/scenario4')
stackedbar_sources(datas['types'], 'Vaccination Status of Infected Individuals by Scenario', save='rerendered/scenario4-vaccine')

In [ ]:
# Scenario 4, verification for statistics.
datas = defaultdict(dict)
for v, ri, ro in [(.5, .3, .3), (.75, .7, .7)]:
    name = 'V' + str(v) + ',' + str((ri, ro))
    sources, distribution, types, summary = get_data('scenario4_' + name)
    totalsd = []
    totalss = []
    for i in range(100):
        totalsd.append(sum([vector[i] for vector in distribution.values()]))
        totalss.append(sum(sources.values()))
    print(np.mean(totalsd), np.median(totalsd), np.mean(totalss))

In [ ]:
# Special graph.  Base model with R0= 2, 3, 4 and Low socialization with R0 = 3.
datas = defaultdict(dict)
for name, label in [('scenario1_r02', 'R0 = 2'),
                    ('scenario1_r03', 'R0 = 3'),
                    ('scenario1_r04', 'R0 = 4'),
                    ('scenario2', 'Low Social, R0 = 3')]:
    sources, distribution, types, summary = get_data(name)
    datas['sources'][label] = deepcopy(sources)
    datas['distribution'][label] = deepcopy(distribution)
    datas['types'][label] = deepcopy(types)

boxplot_totals(datas['distribution'], save='rerendered/baseline_groups')
boxplot_community(datas['distribution'], save='rerendered/baseline_groups')
stackedbar_sources(datas['sources'], 'Source of Infections by Scenario', save='rerendered/baseline_groups')

In [ ]:
# Scenario 5.  There are 3 vaccine efficacy levels that need to be rendered separately.
for rr in [0.2, 0.5, 0.8]:
    datas = defaultdict(dict)
    ri, ro = rr, rr
    for v in [.5, .6, .7, .8, .9, 1.0]:
        name = 'V' + str(v)
        filename = str(rr) + '_' + name
        sources, distribution, types, summary = get_data('scenario5_' + filename)
        datas['sources'][name] = deepcopy(sources)
        datas['distribution'][name] = deepcopy(distribution)
        datas['types'][name] = deepcopy(types)

    filebase = 'rerendered/scenario5_' + str(rr)
    boxplot_totals(datas['distribution'], save=filebase)
    boxplot_community(datas['distribution'], save=filebase)
    stackedbar_sources(datas['sources'], 'Source of Infections by Scenario', save=filebase)
    stackedbar_sources(datas['types'], 'Vaccination Status of Infected Individuals by Scenario', save=filebase + '-vaccine')

In [ ]:
# Scenario 6.  There are 3 vaccine efficacy levels that need to be rendered separately.
for rr in [0.2, 0.5, 0.8]:
    datas = defaultdict(dict)
    ri, ro = rr, rr
    for v in [.5, .6, .7, .8, .9, 1.0]:
        name = 'V' + str(v)
        filename = str(rr) + '_' + name
        sources, distribution, types, summary = get_data('scenario6_' + filename)
        datas['sources'][name] = deepcopy(sources)
        datas['distribution'][name] = deepcopy(distribution)
        datas['types'][name] = deepcopy(types)

    filebase = 'rerendered/scenario6_' + str(rr)
    boxplot_totals(datas['distribution'], save=filebase)
    boxplot_community(datas['distribution'], save=filebase)
    stackedbar_sources(datas['sources'], 'Source of Infections by Scenario', save=filebase)
    stackedbar_sources(datas['types'], 'Vaccination Status of Infected Individuals by Scenario', save=filebase + '-vaccine')